<h1>Tokenization in Modern Language Models</h1>

<h1>Overview</h1>

This post is designed to introduce two prominent tokenization methods, Byte Pair Encoding (BPE) and SentencePiece, which are commonly employed in State-of-the-Art (SOTA) Large Language Models such as the GPT series, ALBERT, and T5.

Initially, I will provide an overview of traditional tokenization techniques, outlining their limitations, and explaining how the more recent methods of BPE and SentencePiece overcome these challenges. Next, I will delve into the underlying concepts behind BPE and SentencePiece, giving you a detailed understanding of how they work. Finally, I'll offer a simple example to demonstrate how these methods can be implemented in Python. To be clear, we won't be coding them from scratch but rather utilizing existing libraries to take advantage of these powerful tokenization techniques.

For those interested in exploring further, the SentencePiece paper can be found <a href='https://arxiv.org/abs/1808.06226'>here</a>.

<h3>Tokenization in Natural Language Processing: A General Overview</h3>


Tokenization is the process of breaking text down into a sequence of tokens. For example, the sentence "The sky is blue" is broken down into ['The', 'sky', 'is', 'blue']. A tokenizer is the tool that implements this tokenization process. Please note that the commonly used term is 'token' rather than 'word,' since tokens may not always correspond to clean English (or any other language) words, such as 'sky' or 'blue'. Instead, you might encounter tokens like '##ve' or 'lo', which may seem meaningless but are quite useful for language models and other NLP applications.

The tokenization process often overlaps with preprocessing tasks like converting all tokens to lowercase, lemmatization or stemming, and removing stop words, among others. As a result of tokenization, one can create a vocabulary, which is the set of all possible tokens in a text corpus. In addition, we will have a tool—the tokenizer—that can break any given text into a sequence of tokens from the vocabulary.



One trivial way of tokenization is simply splitting a text into its constituent words using the spaces between the words:

In [1]:
text = 'The sky is blue'
tokenized_text = text.split(' ')
print(tokenized_text)

['The', 'sky', 'is', 'blue']


Obviously, splitting a text using merely a space is not an optimum way since a text usually includes punctuation, and words might have different forms, etc. For example, given the sentence 'The fact is: the sky has not been blue in the past three days,' should we treat 'is' and 'been' as the same token? Should we convert all letters to lower (upper) case? Should we remove the ':' after the word 'is,' or keep it? These are some natural questions and scenarios that we should answer before tokenizing our texts. Additionally, do not forget that after designing our tokenizer, we should go over all the texts that we have in our corpus and tokenize them to build our vocabulary, which is the set of all tokens in our corpus.

There are a few libraries that can help us perform better tokenization rather than simply splitting the text. In the following, I will give two examples: one using spaCy, and the other using NLTK.

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')
text = "Hello, world! The sky is blue."
doc = nlp(text)
tokens = [token.text for token in doc]
print(tokens)

/Users/lidasafarnejad/PycharmProjects/seq2seqModeling/seq2seqvenv/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


['Hello', ',', 'world', '!', 'The', 'sky', 'is', 'blue', '.']


Please ignore the warning. It simply explains that I am using spaCy version 3.6.1, while the model 'en_core_web_sm' was trained using spaCy version 3.0.0.

The model 'en_core_web_sm' is a versatile pre-trained model that can perform various NLP tasks beyond tokenization. Some of these tasks include Part-of-Speech Tagging, Named Entity Recognition, and Dependency Parsing, to name just three."

Here is another example of tokenization using NLTK:

In [3]:
import nltk
from nltk.tokenize import word_tokenize
text = "Hello, world! The sun is shinning."
tokens = word_tokenize(text)
print(tokens)

['Hello', ',', 'world', '!', 'The', 'sun', 'is', 'shinning', '.']


You may need to download the Punkt tokenizer models using the following line of code:
<i><b>nltk.download('punkt')</b></i>

<h3>How about out-of-vocabulary tokens?</h3>

One immediate question that may arise is how to handle a sentence containing tokens not found in the vocabulary. How should we tokenize it? A common heuristic solution is to replace unseen or unknown tokens with a pre-defined symbol like <UNKNOWN>. However, this approach might overlook tokens that carry significant information, potentially affecting the performance of NLP models. Additionally, dealing with punctuation and special characters often relies on heuristic methods and may vary according to the preferences or standards set by individual developers.

More recently proposed tokenization methods aim to address these limitations by handling out-of-vocabulary words and special tokens or characters in a more standardized and consistent manner.

In what follows, I will delve into BPE and SentencePiece, the two most recent tokenization techniques that are widely employed with state-of-the-art language models.   

<h2>Byte Pair Encoding</h2>

BPE, or Byte-Pair Encoding, is a <i><b>subword</b></i> tokenization method, meaning it breaks down words into smaller units known as subwords. Unlike traditional tokenization, which usually divides texts into individual words, BPE goes a step further by segmenting words into even smaller parts. For instance, the word 'love' might be broken down into 'lo' and 've', and the word 'unhappy' might be segmented into 'un' and 'happy'.

BPE is a data-driven algorithm, meaning that it calculates the frequencies of character pairs and sequences based on the specific data on which it is trained. This ensures that the resulting tokenizer is precisely tailored to the dataset at hand, allowing it to capture unique characteristics within that data.

Unlike methods that rely on a predefined vocabulary, BPE constructs its own vocabulary based on the particular corpus it is trained on. This approach enables the model to detect and capture special patterns within the texts of that corpus. Furthermore, by segmenting words into subwords, BPE has the ability to manage out-of-vocabulary words more effectively. Even words that are not directly present in the vocabulary can be represented through a combination of known subwords, enhancing the model's flexibility and performance.

The BPE algorithm begins by splitting a text into individual characters, treating each one as a separate token. It then trains on the data to iteratively identify and merge the most frequent pairs of characters or sequences. This process continues until a specified number of merges have been achieved, or the desired vocabulary size is met. To gain a clearer understanding, let's examine a simple example.

<h3>Example:</h3>


Suppose we are given a small corpus with the following frequencies: 
<ul>
    <li>low: 5</li>
    <li>lower: 2</li>
    <li>newest: 6 </li>
    <li>widest: 3</li>
</ul>
The BPE algorithm starts by breaking down the words into the characters. Here is the resulting characters and their frequencies:
<ul>
    <li>'l' 'o' 'w': 5</li>
    <li>'l' 'o' 'w' 'e' 'r': 2</li>
    <li>'n' 'e' 'w' 'e' 's' 't': 6 </li>
    <li>'w' 'i' 'd' 'e' 's' 't': 3</li>
</ul>
First, the characters 'e' and 's' appear 9 times together. Thus, we will merge them and continue. Then the new corpus will look like as follows:
<ul>
    <li>'l' 'o' 'w': 5</li>
    <li>'l' 'o' 'w' 'e' 'r': 2</li>
    <li>'n' 'e' 'w' 'es' 't': 6 </li>
    <li>'w' 'i' 'd' 'es' 't': 3</li>
</ul>
Please note we could also choose to merge 's' and 't' because they also appear 9 times together. In such cases, that is when you have <i>ties</i>, you need to have a strategy to handle them. For example, you can randomly choose between them,  you might consider the frequency of indvividual tokens in the pair and choose those with the highest individual tokens, or you might have a predefined order such as alphabetical order.

Second iteration: 'es' and 't' appear 9 times together. Merge them to 'est':
<ul>
    <li>'l' 'o' 'w': 5</li>
    <li>'l' 'o' 'w' 'e' 'r': 2</li>
    <li>'n' 'e' 'w' 'est': 6 </li>
    <li>'w' 'i' 'd' 'est': 3</li>
</ul>

Third iteration: 'l' and 'o' appear 7 times together. Merge them to 'lo':
<ul>
    <li>'lo' 'w': 5</li>
    <li>'lo' 'w' 'e' 'r': 2</li>
    <li>'n' 'e' 'w' 'est': 6 </li>
    <li>'w' 'i' 'd' 'est': 3</li>
</ul>
Fourth Iteration: "lo" and "w" appear together 7 times. Merge them into "low":
<ul>
    <li>'low': 5</li>
    <li>'low' 'e' 'r': 2</li>
    <li>'n' 'e' 'w' 'est': 6 </li>
    <li>'w' 'i' 'd' 'est': 3</li>
</ul>
Fifth Iteration: "n" and "e" appear together 6 times. Merge them into "ne":
<ul>
    <li>'low': 5</li>
    <li>'low' 'e' 'r': 2</li>
    <li>'ne' 'w' 'est': 6 </li>
    <li>'w' 'i' 'd' 'est': 3</li>
</ul>

I believe you've grasped the idea. The vocabulary is $V={'l', 'o', 'w', 'e', 'r', 'n', 'i', 'd', 's', 't', 'es', 'est', 'lo', 'low', 'ne'}$. We can continue until we either reach a predefined vocabulary size or a predefined number of itetions.  


<h3>Implementing BPE</h3>

There certain ways to implement BPE. You can implement it from scratch, or you could use libraries, such as <b>transformers</b>, <b>tokenizers</b>. or <b>SentencePiece</b> from Hugging Face. In the following, I will show you a few examples how to implement your own BPE tokenizer and how to use the libraries. 

<h3>Implementing BPE from scratch</h3>

The code snippet below illustrates a simplified implementation of our example. It's worth noting that a real-world application might be more complex. For instance, we may need to include strategies to manage ties or deal with special characters.

In [4]:
from collections import Counter
import re

def get_stats(vocab):
    pairs = Counter()
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {
    'l o w': 5,
    'l o w e r': 2,
    'n e w e s t': 6,
    'w i d e s t': 3
}

num_merges = 5

for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)

# Printing the final vocabulary
for word, freq in vocab.items():
    print(f'{word}: {freq}')

low: 5
low e r: 2
ne w est: 6
w i d est: 3


<h3>Implementing using libraries</h3>

<b>GPT2Tokenizer from the library transformers from Hugging Face:</b>

The following code snippets show how we can use from the <b>'transformers'</b> library. Please note that if do not have these libraries you can use <i>!pip install transformers</i> to install them. 

In [6]:
#!pip install transformers
from transformers import GPT2Tokenizer

# Load the GPT-2 tokenizer, which uses BPE
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Encode a text string using BPE
text = "Byte-Pair Encoding is a subword tokenization method."
encoded_text = tokenizer.encode(text)
print(f'encoded text: {encoded_text}')
# Decode the encoded text to see how it's been tokenized
decoded_text = tokenizer.decode(encoded_text)
print(f'dencoded text: {decoded_text}')

encoded text: [40778, 12, 47, 958, 14711, 7656, 318, 257, 850, 4775, 11241, 1634, 2446, 13]
dencoded text: Byte-Pair Encoding is a subword tokenization method.


In the code snippet <i><b>GPT2Tokenizer.from_pretrained("gpt2")</i></b>, the identifier 'gpt2' speficies the tokenizer that is designed and configured to work specifically with the GPT-2 language model. It includes the rules and vocabulary necessary to convert text into a numerical format that the GPT-2 model can process. 

<b>ByteLevelBPETokenizer from the tokenizers by Hugging Face:</b>

In [8]:
from tokenizers import ByteLevelBPETokenizer

# Path to the corpus file
corpus_file = "data/PennTreeBank.txt"

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Train the tokenizer on the corpus
tokenizer.train(files=[corpus_file], vocab_size=30_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save the tokenizer
tokenizer.save_model("models")

['models/vocab.json', 'models/merges.txt']

Now, let's print a few lines of 'vocab.json' and 'merges.txt' to see what their contents are.

In [10]:
import json
# Path to vocab and merges files
vocab_path = "models/vocab.json"
merges_path = "models/merges.txt"

# Print a few lines of vocab.json
print("Vocabulary:")
with open(vocab_path, 'r', encoding='utf-8') as vocab_file:
    vocab = json.load(vocab_file)
    for i, (token, idx) in enumerate(vocab.items()):
        print(token, idx)
        if i >= 10: break  # Print only the first 10 items

print("\nMerges:")  # Print a separation line

# Print a few lines of merges.txt
with open(merges_path, 'r', encoding='utf-8') as merges_file:
    for i, line in enumerate(merges_file):
        print(line.strip())
        if i >= 10: break  # Print only the first 10 lines

Vocabulary:
<s> 0
<pad> 1
</s> 2
<unk> 3
<mask> 4
! 5
" 6
# 7
$ 8
% 9
& 10

Merges:
#version: 0.2 - Trained by `huggingface/tokenizers`
Ġ Ġ
Ġ e
Ġ t
Ġ n
Ġ a
Ġ o
Ġ i
Ġ s
Ġ r
Ġ h


<h3>Summary</h3>
<ul>
<li>It is known that the larger the size of the vocabulary, the more powerful the NLP task is as a large vocabular is more comprehensive and more expressive. However, building an NLP system based on a large vocabulary can be computationally expensive.</li>
    <li>By performing subword tokenization, BPE is able to effectively expand the vocabulary size by breaking words into subwords. This also allows it to handle out-of-vocabulary words, as the vocabulary might not include a word directly but likely contains its constituent subwords.</li>
    <li>One issue with the BPE approach, however, is that there might exist several valid subword tokenizations for a single word, as we saw earlier in the example. This ambiguity might impact the performance of the NLP model, as each of these subword representations could be encoded/decoded differently.</li>
</ul>

In the following, I will explain how the SentencePiece framework attempts to address the above challenges or limitations associated with BPE, using unigram language modeling.

<h2>SentencePiece Tokenization</h2>

SentencePiece is a subword tokenization framework proposed in <a href=https://aclanthology.org/D18-2012.pdf >this paper</a> by kudo et. al. At its core, SentencePiece implements two tokenization methods; BPE and unigram language model proposed by kudo in <a href=https://arxiv.org/pdf/1804.10959.pdf>this paper</a>. Earlier in this post, we learned what BPE is, and I will explain how the unigram language model works shortly, providing a code snippet to better grasp how to use it in code. However, before delving into the technical details, let's explore the key advantages of SentencePiece.

Similar to BPE, and in contrast to traditional tokenization methods that simply replace out-of-vocabulary words with a single specific symbol like <UNKNOWN>, SentencePiece can handle out-of-vocabulary words by breaking them down into subwords. SentencePiece is an unsupervised method, meaning it does not need any labeled data for training. It is also data-driven, as it can reserve tokens for special purposes, such as whitespace, based on the specific dataset at hand, rather than using a predefined set of tokens to handle special cases. While it's tailored to the data it's trained on, it doesn't require information specific to a particular language or domain, giving it a degree of data agnosticism. 

As I metioned earlier, SentencePiece can be configured to use both BPE and the unigram language model. When SentencePiece is configured to use BPE, the resulting tokenizer is quite similar to what pure BPE would create. However, SentencePiece can go further by using a few tokens specific to the dataset, and it also handles whitespace as a special character.  

When using the unigram language model, SentencePiece can perform subword regularization during training. Subword regularization introduces randomness into the tokenization process. Unlike conventional static tokenization, where a text is always tokenized the same way, or BPE, which consistently chooses the most frequent pairs to merge, subword regularization allows for randomness by occasionally selecting different valid subword segmentations based on their likelihood for the same sentence.

In the following section, I will explain the concepts and theory behind the unigram language model. This explanation will not only enhance your understanding of how it operates but also highlight the primary differences between the BPE and the unigram language model.  

<h2>Unigram Language Model</h2>

A unigram language model, proposed kudo in <a href=https://arxiv.org/pdf/1804.10959.pdf>this paper</a>, is a subword tokenization method which is able to output multiple segmentations for a word, each with its associated probabilities. In other words, the unigram language model can probabilistically generate different segmentations based on the likelihoods of the subwords, rather than always arriving at a single "best" segmentation. It considers each word or subword as an independent unit and models the probability of each unit based on its frequency in the corpus. It does not perform iterative merging of character pairs like BPE. I know it might sound vague. Thus, before proceeding, I'll provide an example.

<b>Example</b>

Let's say our corpus contains the following sentences:

    "I love to play."
    "I love to sing."
    "I love playing games."
    
The model will then estimate the probabilities for each subword based on its frequency. Assuming an arbitrary breakdown, the probabilities might look something like:

    "I": 0.15
    "love": 0.15
    "to": 0.10
    "play": 0.10
    "ing": 0.05
    "pla": 0.12
    "ying": 0.08
    "sing": 0.05
    "games": 0.05
    
In the context of the unigram language model, the tokenization of a word like "playing" is determined by the probabilities assigned to each possible subword segmentation, and the algorithm will typically choose the segmentation that maximizes the likelihood.

So, in your example:

    If the probabilities for "play" and "ing" are 0.10 and 0.05 respectively, then the likelihood for this segmentation would be 0.10×0.05=0.0050.10×0.05=0.005.
    If the probabilities for "pla" and "ying" are 0.12 and 0.08 respectively, then the likelihood for this segmentation would be 0.12×0.08=0.00960.12×0.08=0.0096.

Since 0.0096 > 0.005, the algorithm would indeed choose the tokenization "pla" and "ying" over "play" and "ing" for the word "playing."

Here are a few notes to consider:
<ul>
        <li>In a unigram language model, each word or subword is treated as an independent unit.</li>
    <li>The probability of a word or subword occurring in the text is modeled solely based on its frequency in the training corpus. It does not depend on the context or the adjacent words.
    This is why it's called a "unigram" model, where "uni" stands for one, meaning that it only considers one word at a time.</li>
</ul>

Now, you may be wondering how these likelihoods are computed during training. In the sections that follow, I will delve into the underlying principles of the unigram language model, providing a concise explanation of how it functions.

<h3>The theory behind the unigram language model</h3>

In this section, I'll break down the mathematical workings of the language model to show what's going on behind the scenes. If you're not into theory and math, feel free to skip right to the coding part.

The unigram language model, uses two algorithms at its core; Expectation-Maximization (EM) and Viterbi, to find the vocabulary consisting of the subwords and segment a given sentence. 


<b>Expectation-Maximization (EM)</b> 

First, given a heuristically generated seed vocabulary, the EM algorithm is used to optimize and determine the probabilities of each subword or segment. This essentially refines the initial guesses of subword probabilities to better fit the data.
As a result, we have an improved vocabulary where each subword or segment has an associated probability.

Given the input sentence $X$ from the corpus $D$, we would like to find a subword tokenization like $x$ such that $x=(x_1,...,x_m)$, where each $x_1,..., x_m$ are subwords which provides one segmenations for the input $X$. Considering the probabilities associated with subwords $x_1,..., x_m$, the probability of $x$, $P(x)$, is then can be formulated as:
$$P(X) = \prod_{i=1}^{m}p(x_i), \forall i\: x_i\in V$$
$$\sum_{x \in V}p(x)=1,$$
The first product means the probability of a word is formulated as the product of the probabilities of its subwords. Please note that the above probability multiplication is valid as the unigram language model assumes that each subwords occurs independently. Additionally, the summation means summing up the probability of all the tokens in the vocabulary $V$ equals one. 

Assume we are provided with a vocabulary $V$. We can then estimate the probabilities of individual subword occurrences, $p(x_i)$, using the EM algorithm. This estimation is done by maximizing the marginal likelihood $L$, where $p(x_i)$ is treated as a hidden variable. The formula for $L$ is

However, in practical scenarios, we do not have the vocabulary in advance. The authors of <a href=https://arxiv.org/pdf/1804.10959.pdf>the paper</a> explains that we can start with an initial seed vocabulary and then refine it by iteratively performing the following steps until the vocabulary size reaches a specified target:
<ol>
    <li> With the current vocabulary fixed, refine the subword probabilities using the EM algorithm.</li>
<li>Determine the impact (loss) of each subword on the likelihood LL if it were to be removed from the vocabulary</li>
<li>Organize the subwords based on their loss values and retain only the top fraction (e.g., the top 80%). It's important to always include single-character subwords to prevent words from being completely absent in our vocabulary.</li>
    </ol>



<h3>Example</h3>

<b>Vocabulary Initialization</b>

Assume our initial seed vocabulary (a large set of possible subwords) is:
<ul>
    <li>V = {"p", "l", "a", "y", "i", "n", "g", "pla", "ying", "play", "ing", "am", "with"}</li>
</ul>
We can use the corpus to find the frequencies and compute the initial probabilities. 

<b>(a) Optimize $p(x)$ with the EM Algorithm:</b>
    
We will use the EM algorithm to optimize the probabilities of these subwords, but for the purpose of this example, let's simplify and assume that after running the EM algorithm, the probabilities have been updated to:
<ul>
<li>p("am") = 0.03, p("with") = 0.02, p("p") = 0.05, p("l") = 0.05, p("a") = 0.05, p("y") = 0.05, p("i") = 0.05, p("n") = 0.05, p("g") = 0.05, p("pla") = 0.20, p("ying") = 0.20, p("play") = 0.15, p("ing") = 0.10</li>
</ul>
    
<b>(b) Compute the $loss_i$ for each subword $x_i$:</b>
    
Next, we want to compute how likely the likelihood $L$ is reduced if a subword is removed. Let's say the loss is computed as the difference between the current likelihood and the likelihood after removing the subword (more complex calculations might be used in practice). We might get:
    <ul>
<li>loss("p") = 0.01, loss("l") = 0.01, loss("a") = 0.01, loss("y") = 0.01, loss("i") = 0.01, loss("n") = 0.01, loss("g") = 0.01, loss("pla") = 0.05, loss("ying") = 0.04, loss("play") = 0.03, loss("ing") = 0.02, loss("with") = 0.003, loss("am") = 0.005,</li>
    </ul>
<b>(c) Sort the Symbols by $loss_i$ and Keep Top $η%$ of subwords:</b>
    
Next, we sort the subwords by loss:
<ul>
    
    <li>"pla", "ying", "play", "ing", "p", "l", "a", "y", "i", "n", "g", "am", "with"</li>
</ul>
    
If $η = 80%$, we'll retain the top $80%$ of these subwords, excluding the single characters to avoid out-of-vocabulary issues:
<ul>    
    <li>V_new = {"pla", "ying", "play", "ing", "p", "l", "a", "y", "i", "n", "g"}</li>
</ul>
  


<b>Viterbi Algorithm</b>:
Once we have this refined vocabulary with probabilities (from the EM algorithm), the Viterbi algorithm is then employed for a given sentence, $X$, to traverse through the possible segmentations of X and find the most probable segmentation based on these probabilities.
Essentially, Viterbi helps decide how to split a given sentence using the segments from the vocabulary in a way that maximizes the overall probability of the segmentation.

The best subword sequence, $x^*$, is the one that maximizes the probability $P(X)$. In other words,
$$x^∗ =argmax_{x \in S(X)}P(x), $$
where $S(x)$ is a set of segmenation candidates built for the input sentence $X$. 

In [ ]:
import torch
from torchtext.datasets import PennTreebank
import sentencepiece as spm

train_data, valid_data, test_data = PennTreebank()

with open('data/PennTreeBank.txt', 'w') as f:
    for dataset in [train_data, valid_data, test_data]:
        for sentence in dataset:
            f.write(' '.join(sentence) + '\n')
spm.SentencePieceTrainer.Train('--input=data/PennTreeBank.txt --model_prefix=models/SentencePiecePennTree --vocab_size=67')


The most probable segmentation $x^∗$ for the input sentence $X$ is then given by
$$x^∗ =argmaxP(x), x \in S(X)$$
where $S(X)$ is a set of segmentation candidates built from the input sentence $X$. $x^∗$ is obtained with the Viterbi algorithm (Viterbi, 1967).
If the vocabulary $V$ is given, subword occurrence probabilities $p(x_i)$ are estimated via the EM algorithm that maximizes the following marginal likelihood $L$ assuming that $p(x_i)$ are hidden variables.
$$L = \sum_{s=1}^{|D|}log(P(X^s)) = \sum_{s=1}^{|D|}log(\sum_{x \in S(X^{(s)})}P(X))$$
In the real setting, however, the vocabulary set V is also unknown. Because the joint optimization of vocabulary set and their occurrence probabili- ties is intractable, we here seek to find them with the following iterative algorithm.
1. Heuristically make a reasonably big seed vo- cabulary from the training corpus.
2. Repeat the following steps until |V| reaches a desired vocabulary size.
(a) Fixing the set of vocabulary, optimize p(x) with the EM algorithm.
(b) Compute the lossi for each subword xi, where lossi represents how likely the likelihood L is reduced when the sub- word xi is removed from the current vo- cabulary.
(c) Sort the symbols by lossi and keep top η % of subwords (η is 80, for example). Note that we always keep the subwords consisting of a single character to avoid out-of-vocabulary.
